# Training BYOC on SageMaker
>__Note:__ This assumes a container has already been built and deployed to ECR.

## Prepare the Data

In [ ]:
# Load the required libraries
import warnings
import zipfile
import boto3
import os
import json
import urllib.request
import sagemaker
import tempfile
import cv2
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
from sagemaker.estimator import Estimator
from sklearn.model_selection import train_test_split

In [ ]:
# Configure SageMaker parameters
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

## Set Hyperparameters
>__Note:__ Hyperparameters to be supplied by instructor are:
1. `data_bucket`: `s3://sagemaker-workshop-500842391574-us-east-1`
2. `training_image`: `500842391574.dkr.ecr.us-east-1.amazonaws.com/pystig:keras-gpu`
3. `hosting_image`: `500842391574.dkr.ecr.us-east-1.amazonaws.com/pystig:keras-cpu`

In [ ]:
# Configure the hyperparameters from the instructor
data_bucket = '<<PROVIDED BY INSTRUCTOR>>'
training_image = '<<PROVIDED BY INSTRUCTOR>>'
hosting_image = '<<PROVIDED BY INSTRUCTOR>>'

# Training data channel
channels = {'train': data_bucket}

# Optmized training parameters
hyperparameters = dict(batch_size=32, learning_rate=.0001, epochs=12)

# Output of trained model
output_location = "s3://{}".format(bucket)

# SageMaker estimator
BYOC_estimator = Estimator(
    training_image,
    role=role,
    output_path=output_location,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters,
    sagemaker_session=sagemaker_session
)

# Start training
BYOC_estimator.fit(channels)

## Training Job Description
>__Note:__ Make sure to remember the name of the training job above.

In [ ]:
# Add name of the training job
job_name = '<<TRAINING JOB NAME>>'
response = sagemaker_client.describe_training_job(
    TrainingJobName=job_name
)
response

---
## Deploy model - Standard
__Use `estimator.deploy()` based on GPU Container training__
>__Note:__ This is not cost effective as it uses the GPU training image, plus it causes complications if not using th

```
predictor = BYOC_estimator.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')
```

---
## Deploy Model - CPU Container
__Use separate CPU container and the `sagemaker.Session()` API to specify a different serving container__

### Step 1: Create a new model from the training job, specifying a different container for training, in this case a CPU-based container.

In [ ]:
BYOC_model = sagemaker_session.create_model_from_job(
    name = job_name.split('-')[0]+'-model',
    training_job_name=job_name,
    role=role,
    primary_container_image=hosting_image,
    model_data_url='s3://{}/{}/output/model.tar.gz'.format(bucket, job_name)
)

### Step 2: Create a SageMaker Endpoint Configuration

In [ ]:
BYOC_endpoint_config_name = sagemaker_session.create_endpoint_config(
    name=job_name.split('-')[0]+'-endpoint-config',
    model_name=BYOC_model,
    initial_instance_count=1,
    instance_type='ml.c4.xlarge'
)

### Step 3: Deploy the SageMaker Endpoint

In [ ]:
create_endpoint_response = sagemaker_session.create_endpoint(
    endpoint_name=job_name.split('-')[0]+'-endpoint',
    config_name=str(BYOC_endpoint_config_name)
)

In [ ]:
sagemaker_client.describe_endpoint(EndpointName=create_endpoint_response)

## Test Endpoint using Random Test Image using `numpy` Arrays (Simuilate pyStig driver)

<details><summary><strong>Note to self</strong></summary><p>
    DO NOT DOWNLOAD `data.zip`. Use the `numpy` arrays in `/tmp/`
    </p>
</details>

In [ ]:
test_image = np.load('/tmp/data/valid_X.npy')[42]
steering_truth = np.load('/tmp/data/valid_Y.npy')[42]

# Image shape
test_image.shape

In [ ]:
from PIL import Image
Image.fromarray(test_image.astype('uint8'))

In [ ]:
# Simulate pyStig call by first pre-preocessing image and converting to 4D array
endpoint_name = sagemaker_client.describe_endpoint(EndpointName=create_endpoint_response)['EndpointName']
payload = test_image.reshape(1, 66, 200, 3)

# Confirm image shape
payload.shape

In [ ]:
# Invoke SageMaker endpoint with image data
runtime_client = boto3.client('sagemaker-runtime')
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload.tolist())
)
steering_prediction = float(json.loads(response['Body'].read().decode('utf-8'))[0])

# Predicted steering angle prediction on image 42
steering_prediction

In [ ]:
steering_truth